In [183]:
# coding=utf-8
import math
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd

hosts = dict([('www.easternct.edu','149.152.33.32'), ('web.mit.edu','23.197.229.238')])

DEFAULT_RTT_VALUE = 0

ATTEMPS = 30

def enviar_paquete(host, time_to_live):
    packet = IP(dst=host, ttl=time_to_live) / ICMP()
    res = sr(packet, timeout=1, verbose=0)
    return res[0][ICMP]

In [184]:
def trace_route(hosts):
    
    hops = []
    
    for host in hosts:
        ttl_without_response = []
        ip = hosts[host]
        tipo = 11
        ttl = 1
        while (tipo!=0 and ttl < 30):
            for i in range(0,ATTEMPS):
                if (not ttl in ttl_without_response):
                    ti = time.time()
                    res1 = enviar_paquete(ip, ttl)
                    tf = time.time()
                    rtt = tf-ti
                    if (len(res1) > 0):
                        tipo = res1[0][1].type
                        src = res1[0][1].src
                        if (tipo > 0):
                            hops.append([host,ttl,i,src,rtt])
                        else:
                            hops.append([host,ttl,i,src,rtt])
                    else:
                        hops.append([host,ttl,i,None,DEFAULT_RTT_VALUE])
                        ttl_without_response.append(ttl)
            ttl = ttl + 1

    hops_info = pd.DataFrame(hops,columns=['host','ttl','i','ip','rtt'])
    return hops_info

In [187]:
df = trace_route(hosts)
df_indexed = df.set_index(['host','ttl','i'])
df_indexed

ip       rtt
host              ttl i                           
web.mit.edu       1   0             None  0.000000
                  2   0             None  0.000000
                  3   0             None  0.000000
                  4   0             None  0.000000
                  5   0             None  0.000000
                  6   0   200.89.161.101  0.063950
                      1   200.89.161.101  0.067979
                      2   200.89.161.101  0.068022
                      3   200.89.161.101  0.067797
                      4   200.89.161.101  0.052200
                      5   200.89.161.101  0.083740
                      6   200.89.161.101  0.075924
                      7   200.89.161.101  0.055952
                      8   200.89.161.101  0.155953
                      9   200.89.161.101  0.075816
                      10  200.89.161.101  0.079881
                      11  200.89.161.101  0.067870
                      12  200.89.161.101  0.083911
                      13  200.89.161.101  0.076014
                      14  200.89.161.101  0.087813
                      15  200.89.161.101  0.107785
                      16  200.89.161.101  0.103959
                      17  200.89.161.101  0.087885
                      18  200.89.161.101  0.139920
                      19  200.89.161.101  0.139921
                      20  200.89.161.101  0.095932
                      21  200.89.161.101  0.080024
                      22  200.89.161.101  0.079901
                      23  200.89.161.101  0.076033
                      24  200.89.161.101  0.067719
...                                  ...       ...
www.easternct.edu 16  13   72.10.110.141  0.220069
                      14   72.10.110.141  0.212021
                      15   72.10.110.141  0.207674
                      16   72.10.110.141  0.207940
                      17   72.10.110.141  0.203897
                      18   72.10.110.141  0.211910
                      19   72.10.110.141  0.211964
                      20   72.10.110.141  0.219846
                      21   72.10.110.141  0.203929
                      22   72.10.110.141  0.211910
                      23   72.10.110.141  0.216045
                      24   72.10.110.141  0.211777
                      25   72.10.110.141  0.224052
                      26   72.10.110.141  0.227916
                      27   72.10.110.141  0.223746
                      28   72.10.110.141  0.212105
                      29   72.10.110.141  0.199774
                  17  0             None  0.000000
                  18  0             None  0.000000
                  19  0             None  0.000000
                  20  0             None  0.000000
                  21  0             None  0.000000
                  22  0             None  0.000000
                  23  0             None  0.000000
                  24  0             None  0.000000
                  25  0             None  0.000000
                  26  0             None  0.000000
                  27  0             None  0.000000
                  28  0             None  0.000000
                  29  0             None  0.000000

[277 rows x 2 columns]

In [121]:
import logging
import sys
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
dst_ip = "web.mit.edu"
src_port = 400
dst_port=80
 
tcp_connect_scan_resp = sr1(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="S"),timeout=10)
if(tcp_connect_scan_resp is None):
    print ("The port is Closed")
    sys.exit()
     
print ("The flags:" + str (tcp_connect_scan_resp.getlayer(TCP).flags))    
    
if(tcp_connect_scan_resp.haslayer(TCP)):
    if(tcp_connect_scan_resp.getlayer(TCP).flags == 0x12):
        #send_rst = sr(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="AR"),timeout=3)
        print ("The port is Open")
        print tcp_connect_scan_resp.getlayer(IP).src
        sys.exit ();
            
#(tcp_connect_scan_resp.getlayer(TCP).flags == 0x14):
print ("The port is Closed ")

Begin emission:
Finished to send 1 packets.

Received 46 packets, got 1 answers, remaining 0 packets
The flags:18
The port is Open
23.197.229.238


SystemExit: 

In [151]:
df.set_index?


In [41]:
sr1(IP(dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname="www.google.com", qclass="IN", qtype="A")),verbose=0)

<IP  version=4L ihl=5L tos=0x0 len=76 id=14325 flags= frag=0L ttl=55 proto=udp chksum=0x7990 src=8.8.8.8 dst=192.168.1.100 options=[] |<UDP  sport=domain dport=domain len=56 chksum=0x92bb |<DNS  id=0 qr=1L opcode=QUERY aa=0L tc=0L rd=1L ra=1L z=0L ad=0L cd=0L rcode=ok qdcount=1 ancount=1 nscount=0 arcount=0 qd=<DNSQR  qname='www.google.com.' qtype=A qclass=IN |> an=<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=291 rdata='172.217.29.4' |> ns=None ar=None |>>>

In [81]:
import dns.resolver
my_resolver = dns.resolver.Resolver()
my_resolver.nameservers = ['8.8.8.8']

answer=my_resolver.query("google.com", "A")
for data in answer:
    print data.address

172.217.28.238


In [82]:
dns.rdtypes.IN.A.A.address

<member 'address' of 'A' objects>